In [1]:
import pymongo
from pymongo import MongoClient
import tqdm
from collections import defaultdict, Counter
import json

client = MongoClient('localhost', 27017)
db = client.istex
cambridge = db.cambridge
springer = db.springer
total = springer.find({'categories': {'$exists': True}, 'abstract': {'$exists': True}, 'title': {'$exists': True}}).count() + cambridge.find({'categories': {'$exists': True}, 'abstract': {'$exists': True}, 'title': {'$exists': True}}).count()
categories = defaultdict(set)

cnt = Counter()

with open('springer_cambridge.txt', 'a') as outfile:
    for col in [springer, cambridge]:
        for i in tqdm.tqdm(col.find({'categories': {'$exists': True}, 'abstract': {'$exists': True}, 'title': {'$exists': True}})):
            for ar, v in i['categories'].iteritems():
                if ar == 'scopus':
                    cat = set()
                    for el in v:
                        if '1 -' in el:
                            cat.add(el)
                    cat = list(cat)
                    json.dump({'abstract': i['abstract'], 'label': ','.join(cat), 'id': i['id'], 'title': i['title']}, outfile)
                    outfile.write('\n')


61880it [00:37, 1646.40it/s]
96661it [00:51, 1891.15it/s]


In [2]:
import pandas as pd

d = pd.read_json("springer_cambridge.txt", orient='records', lines=True)
d = d[d.label != '']

In [3]:
d.count()

abstract    93596
id          93596
label       93596
title       93596
dtype: int64

In [4]:
d.labels = d.label.apply(lambda x: x.split(','))

In [5]:
from collections import Counter
r = []
di = set()

for l in d.labels:
    lab_di = {'Health Sciences':0, 'Life Sciences':0, 'Physical Sciences':0, 'Social Sciences':0}
    for i in l:
        lab_di[i[4:]] += 1
    r.append(lab_di)
d['labels_dict'] = pd.Series(r)

In [6]:
d_nonan = d.dropna(axis=0, how='any')

In [7]:
hs = []
ls = []
ps = []
ss = []

nd = {'Health Sciences': hs, 'Life Sciences': ls, 'Physical Sciences': ps, 'Social Sciences': ss}

for l in d_nonan.labels_dict.tolist():
    nd['Health Sciences'].append(l['Health Sciences'])
    nd['Life Sciences'].append(l['Life Sciences'])
    nd['Physical Sciences'].append(l['Physical Sciences'])
    nd['Social Sciences'].append(l['Social Sciences'])
nd

{'Health Sciences': [0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
 

In [8]:
d_nonan['health_sciences'] = nd['Health Sciences']
d_nonan['life_sciences'] = nd['Life Sciences']
d_nonan['physical_sciences'] = nd['Physical Sciences']
d_nonan['social_sciences'] = nd['Social Sciences']

d_nonan.head()

/home/sami/.virtualenvs/jupyter/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/sami/.virtualenvs/jupyter/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sami/.virtualenvs/jupyter/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,abstract,id,label,title,labels_dict,health_sciences,life_sciences,physical_sciences,social_sciences
0,Abstract: Uptake and metabolic turnover of a v...,91543AB49EF6104156B755A81D4D3E11B1E3CF19,"1 - Physical Sciences,1 - Life Sciences",Activity of marine psychrophilic bacteria at e...,"{u'Physical Sciences': 1, u'Life Sciences': 1,...",0,1,1,0
1,"Summary: The calcium antagonist falipamil, a c...",EFB951031543D6D1B8479250A5B6BBAB6A7B24C8,"1 - Life Sciences,1 - Health Sciences",Atropine and verapamil interactions in healthy...,"{u'Physical Sciences': 0, u'Life Sciences': 1,...",1,1,0,0
2,Summary: Experiments were performed to compare...,9DEA4ED385B3B941293CBADB71A3004AA4BFA390,1 - Life Sciences,Mechanism of asymmetric block of K channels by...,"{u'Physical Sciences': 0, u'Life Sciences': 1,...",0,1,0,0
3,Abstract: Cells of the coccolithophoridCricosp...,152CF0B3C7000B08F84612084EE4CC844F692482,"1 - Physical Sciences,1 - Life Sciences",Ultrastructural study and calcium and cadmium ...,"{u'Physical Sciences': 1, u'Life Sciences': 1,...",0,1,1,0
4,Abstract: A calcium requirement was shown for ...,0866A00EDFAA77712E45BCCAFEFC042E296D3772,"1 - Life Sciences,1 - Health Sciences",The calcium requirement for functional vesicle...,"{u'Physical Sciences': 0, u'Life Sciences': 1,...",1,1,0,0


In [9]:
d_nonan = d_nonan.drop("labels_dict", axis=1)

In [10]:
d_nonan.to_csv('scopus.csv', encoding='utf-8', index=False)

In [11]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(d_nonan, test_size=0.2)

In [15]:
train.to_csv('scopus_train.csv', encoding='utf-8', index=False)
test.to_csv('scopus_test.csv', encoding='utf-8', index=False)

print train.shape
print test.shape

(74795, 8)
(18699, 8)
